In [ ]:
# Import all necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from keras.layers import Dense
from keras.models import Sequential
from keras import losses 
from keras.datasets import mnist
import keras
import seaborn as sns
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from tqdm import tqdm
import os
import cv2
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Mount google drive content

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Unzip FaceMaskDetection dataset

!unzip "/content/drive/MyDrive/Datasets/FaceMaskDetection.zip"

In [ ]:
# Setting paths for two classes

with_mask = "/content/data/with_mask"
without_mask = "/content/data/without_mask"

In [ ]:
# Pack image data to np array

with_mask_data = list()
for file in tqdm(os.listdir(with_mask),position=0):
    path  = os.path.join(with_mask,file)
    image = cv2.imread(path)
    image = cv2.resize(image,(64,64))
    image = image.tolist()
    with_mask_data.append(image)

100%|██████████| 3725/3725 [00:25<00:00, 145.96it/s]


In [ ]:
# Pack image data to np array

without_mask_data = list()
for file in tqdm(os.listdir(without_mask),position=0):
    path = os.path.join(without_mask,file)
    image = cv2.imread(path)
    image = cv2.resize(image,(64,64))
    image = image.tolist()
    without_mask_data.append(image)

100%|██████████| 3828/3828 [00:17<00:00, 219.21it/s]


In [ ]:
with_mask_data = np.array(with_mask_data)
without_mask_data = np.array(without_mask_data)
X = np.concatenate((with_mask_data,without_mask_data), axis = 0)
Y = np.concatenate((np.ones(3725), np.zeros(3828)), axis = 0)

In [ ]:
# Setting different train and test data

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=42)

In [ ]:
# Print train and test data np array shape

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(5664, 64, 64, 3)
(1889, 64, 64, 3)
(5664,)
(1889,)


In [ ]:
# Face Mask Detection using CNN

input_shape = (64, 64, 3)
model = Sequential()
model.add(Conv2D(filters=64, kernel_size = (3,3), activation='relu',input_shape=input_shape,data_format='channels_last'))
model.add(Conv2D(filters=64, kernel_size = (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())    
model.add(Conv2D(filters=256, kernel_size = (3,3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(512,activation="relu"))
model.add(Dense(128,activation='sigmoid'))
model.add(Dense(1,activation="sigmoid"))

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 30, 30, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 30, 30, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 128)       73856     
                                                                 
 conv2d_3 (Conv2D)           (None, 26, 26, 128)      

In [ ]:
model.compile(optimizer='sgd',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True
)

In [ ]:
train_gen = datagen.flow(X_train,y_train,batch_size=32)
test_gen = datagen.flow(X_test,y_test,batch_size=32)

In [ ]:
history = model.fit(train_gen,
                              epochs=20,
                              steps_per_epoch = X_train.shape[0] // 32
                              ,verbose=1,
                              validation_data=test_gen,
                              validation_steps = X_test.shape[0] // 32)

Epoch 1/20
177/177 [==============================] - 303s 2s/step - loss: 0.3239 - accuracy: 0.8683 - val_loss: 0.4236 - val_accuracy: 0.7929
Epoch 2/20
177/177 [==============================] - 304s 2s/step - loss: 0.2586 - accuracy: 0.8939 - val_loss: 0.1925 - val_accuracy: 0.9306
Epoch 3/20
177/177 [==============================] - 306s 2s/step - loss: 0.2249 - accuracy: 0.9145 - val_loss: 0.1954 - val_accuracy: 0.9259
Epoch 4/20
177/177 [==============================] - 305s 2s/step - loss: 0.2118 - accuracy: 0.9153 - val_loss: 0.1822 - val_accuracy: 0.9341
Epoch 5/20
177/177 [==============================] - 304s 2s/step - loss: 0.1840 - accuracy: 0.9290 - val_loss: 0.2609 - val_accuracy: 0.8824
Epoch 6/20
177/177 [==============================] - 303s 2s/step - loss: 0.1713 - accuracy: 0.9333 - val_loss: 0.4828 - val_accuracy: 0.7294
Epoch 7/20
177/177 [==============================] - 303s 2s/step - loss: 0.1551 - accuracy: 0.9428 - val_loss: 0.2298 - val_accuracy: 0.9153

In [ ]:
model.evaluate(test_gen,verbose=1)

60/60 [==============================] - 26s 432ms/step - loss: 0.1107 - accuracy: 0.9614


[0.11066458374261856, 0.9613552093505859]

In [ ]:
model.save("FaceMaskModel")

INFO:tensorflow:Assets written to: FaceMaskModel/assets


In [ ]:
# model.save('my_model.h5')

In [ ]:
model = keras.models.load_model("/content/FaceMaskModel")

In [ ]:
model.evaluate(test_gen,verbose=1)[1]

60/60 [==============================] - 26s 432ms/step - loss: 0.1053 - accuracy: 0.9619


0.961884617805481

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving haarcascade_frontalface_default.xml to haarcascade_frontalface_default.xml


In [ ]:
import shutil
shutil.make_archive("FaceMask", 'zip', "/content/FaceMaskModel")

'/content/FaceMask.zip'

In [ ]:
# import joblib 
# joblib.dump(model,"FaceMaskDetectionModel.sav")

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX 
  
# org 
org = (50, 50) 
  
# fontScale 
fontScale = 0.5
   
# Blue color in BGR 
color = (255, 0, 0) 
  
# Line thickness of 2 px 
thickness = 2

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
video_capture = cv2.VideoCapture(0)

while True:
    # Capture frame-by-frame
    ret, frame = video_capture.read()
  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(30, 30),
        flags=cv2.CASCADE_SCALE_IMAGE
    )

    # Draw a rectangle around the faces
    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        extracted_face = frame[y:y+h, x:x+w]
        image = cv2.resize(extracted_face,(64,64))
        image1 = np.array([image])
        value = model.predict(image1)[0][0]
        accuracy = model.predict(image1)[0][0]
        class_name = ""
        if(value==0):
            class_name = "Without Mask"
        elif(value==1):
            class_name = "With Mask"
        text = "{} ({})".format(class_name,accuracy)
        final_image = cv2.putText(extracted_face,text,org,font,fontScale, color, thickness, cv2.LINE_AA)

    # Display the resulting frame
    cv2.imshow('Video', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything is done, release the capture
video_capture.release()
cv2.destroyAllWindows()
